# 🎯 Stage 1.5 - Latent Separability Audit

## Auditoria de Separabilidade Latente (Accent × Speaker) em Backbones TTS

**Hardware**: Google Colab L4 GPU (24GB VRAM)  
**Tempo estimado**: ~2-3 horas para pipeline completo  
**Autor**: OpenCode Research Lab

---

### 📋 O que este notebook faz:

1. ✅ **Setup**: Instala dependências e clona repositório
2. ✅ **Dataset**: Baixa/prepara dados de áudio (3 regiões brasileiras)
3. ✅ **Features**: Extrai representações do backbone TTS (Qwen3-TTS)
4. ✅ **Probes**: Treina classificadores lineares (Accent × Speaker)
5. ✅ **Analysis**: Gera heatmaps e relatório com decisão GO/NOGO

### ⚡ Otimizações para L4:

- **Mixed precision**: bfloat16 (reduz VRAM em 50%)
- **Flash Attention 3**: kernels otimizados para L4
- **Batch processing**: pipeline em lotes
- **Cache management**: limpa VRAM entre etapas

---

### 🚀 Início Rápido:

1. **Runtime** → Change runtime type → **L4 GPU**
2. Execute todas as células sequencialmente
3. Aguarde ~2h para completar
4. Baixe `report/stage1_5_report.md` no final


---

## 🔧 Parte 1: Setup e Instalação

Instala todas as dependências e aplica os fixes críticos.


In [ ]:
# Verificar GPU disponível
!nvidia-smi -L
!nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits

import torch
print(f"\n🎮 GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
print(f"✅ CUDA: {torch.version.cuda}")
print(f"✅ PyTorch: {torch.__version__}")

In [ ]:
%%time
# Instalar dependências (~ 5 min)
print("📦 Instalando dependências...\n")

# Instalar PyTorch com CUDA 12.1 (otimizado para L4)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Instalar Qwen3-TTS
!pip install -q -U qwen-tts

# Instalar Flash Attention 3 (kernels otimizados para L4)
!pip install -q flash-attn --no-build-isolation

# Dependências do Stage 1.5
!pip install -q numpy scipy pandas scikit-learn librosa praat-parselmouth soundfile
!pip install -q speechbrain transformers datasets huggingface-hub sentence-transformers
!pip install -q hydra-core omegaconf tqdm typer pyyaml jinja2
!pip install -q matplotlib seaborn

print("\n✅ Dependências instaladas!")

In [ ]:
%%time
# Clonar repositório Stage 1.5 (~ 30 sec)
import os

REPO_URL = "https://github.com/seu-usuario/stage1_5.git"  # ⚠️ EDITAR COM SEU REPO

if not os.path.exists("/content/stage1_5"):
    print(f"📥 Clonando {REPO_URL}...")
    !git clone -q {REPO_URL} /content/stage1_5
    print("✅ Repositório clonado!")
else:
    print("✅ Repositório já existe")

%cd /content/stage1_5

In [ ]:
%%time
# Aplicar fixes críticos (~ 10 sec)
print("🔧 Aplicando correções críticas...\n")

# Fix 1: Type hints no adapter
!sed -i 's/def prepare_inputs(self, entry: ManifestEntry, text: str) -> Dict\[str, torch.Tensor\]:/def prepare_inputs(self, entry: ManifestEntry, text: str) -> Dict[str, Any]:/' stage1_5/backbone/huggingface.py

# Fix 2: Forward method (aplicar patch completo)
fix_forward = '''
    def forward(self, inputs: Dict[str, Any]) -> torch.Tensor:
        """Execute forward pass. For Qwen3-TTS, inputs contains generation params."""
        with torch.no_grad():
            if self._model_type == "qwen3_tts":
                mode = inputs.get("mode", "custom_voice")
                if mode == "custom_voice":
                    self.model.generate_custom_voice(
                        text=inputs["text"],
                        language=inputs.get("language", "Portuguese"),
                        speaker=inputs.get("speaker", "ryan"),
                        instruct=inputs.get("instruct"),
                        non_streaming_mode=True,
                        max_new_tokens=inputs.get("max_new_tokens", 256),
                    )
                    return torch.empty(0)
                raise ValueError(f"Unsupported qwen3_tts mode: {mode}")
            return self.model(**inputs)
'''

# Aplicar via Python (mais seguro que sed)
with open("stage1_5/backbone/huggingface.py", "r") as f:
    content = f.read()

# Substituir método forward
import re
content = re.sub(
    r'def forward\(self, inputs\):.*?return self\.model\(\*\*inputs\)',
    fix_forward.strip(),
    content,
    flags=re.DOTALL
)

with open("stage1_5/backbone/huggingface.py", "w") as f:
    f.write(content)

# Fix 3: Layer resolution (adicionar método se não existir)
fix_resolve = '''
    def resolve_layer(self, alias: str) -> Optional[torch.nn.Module]:
        """Resolve layer alias to module."""
        if self._model_type != "qwen3_tts":
            modules = dict(self.model.named_modules())
            return modules.get(alias)
        
        modules = dict(self.model.named_modules())
        if alias in modules:
            return modules[alias]
        
        aliases_map = {
            "text_encoder_out": "talker.text_projection",
            "pre_vocoder": "talker.codec_head",
        }
        
        if alias in aliases_map:
            candidate = aliases_map[alias]
            if candidate in modules:
                return modules[candidate]
        
        if alias.startswith("decoder_block_"):
            suffix = alias.split("decoder_block_", 1)[1]
            if suffix.isdigit():
                idx = int(suffix)
                candidate = f"talker.model.layers.{idx}"
                if candidate in modules:
                    return modules[candidate]
        
        return None
'''

if "def resolve_layer" not in content:
    # Adicionar antes do último método
    content = content.replace(
        "\nclass HuggingFaceBackboneAdapter:",
        f"\nclass HuggingFaceBackboneAdapter:\n{fix_resolve}"
    )
    with open("stage1_5/backbone/huggingface.py", "w") as f:
        f.write(content)

print("✅ Fixes aplicados!")
print("✅ Instalando pacote...")

# Instalar em modo editável
!pip install -q -e .

print("\n🎉 Setup completo!")

---

## 📊 Parte 2: Preparação do Dataset

Baixa e prepara dataset de áudio com 3 regiões brasileiras.


In [ ]:
%%time
# Opção 1: Usar dataset público (recomendado)
# Baixar Common Voice Brasil ou similar

import os

# ⚠️ EDITAR: URL do seu dataset (ZIP com áudios + metadata.csv)
DATASET_URL = "https://exemplo.com/dataset_stage1_5.zip"

# Ou usar dataset sintético para testes
USE_SYNTHETIC = True  # Mudar para False para usar dataset real

if USE_SYNTHETIC:
    print("🧪 Gerando dataset sintético para teste...\n")
    
    # Criar estrutura de diretórios
    !mkdir -p data/wav data/splits
    
    # Gerar áudios sintéticos
    import numpy as np
    import soundfile as sf
    
    accents = ["NE", "SE", "S"]
    speakers_per_accent = 3
    texts_per_speaker = 10
    
    sr = 16000
    duration = 2
    
    manifest_rows = []
    text_rows = []
    
    for accent in accents:
        for spk_idx in range(speakers_per_accent):
            speaker = f"spk{accent}{spk_idx:02d}"
            spk_dir = f"data/wav/{speaker}"
            os.makedirs(spk_dir, exist_ok=True)
            
            for text_idx in range(texts_per_speaker):
                text_id = f"t{text_idx:02d}"
                utt_id = f"{speaker}_{accent}_{text_id}"
                
                # Gerar áudio (ruído branco simples)
                audio = np.random.randn(sr * duration).astype(np.float32) * 0.1
                wav_path = f"{spk_dir}/{text_id}.wav"
                sf.write(wav_path, audio, sr)
                
                # Adicionar ao manifest
                manifest_rows.append({
                    "utt_id": utt_id,
                    "path": wav_path,
                    "speaker": speaker,
                    "accent": accent,
                    "text_id": text_id,
                    "source": "real"
                })
                
                # Adicionar texto
                if {"text_id": text_id, "text": f"Texto de exemplo número {text_idx}"} not in text_rows:
                    text_rows.append({
                        "text_id": text_id,
                        "text": f"Texto de exemplo número {text_idx} para teste de separabilidade"
                    })
    
    # Salvar manifest
    import json
    with open("data/manifest.jsonl", "w") as f:
        for row in manifest_rows:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")
    
    # Salvar textos
    with open("data/texts.json", "w") as f:
        json.dump(text_rows, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Dataset sintético criado:")
    print(f"   - {len(manifest_rows)} utterances")
    print(f"   - {len(accents)} accents: {accents}")
    print(f"   - {accents.count() * speakers_per_accent} speakers")
    print(f"   - {len(text_rows)} unique texts")

else:
    print(f"📥 Baixando dataset de {DATASET_URL}...\n")
    !mkdir -p data
    !wget -q {DATASET_URL} -O data/dataset.zip
    !unzip -q data/dataset.zip -d data/
    
    # Construir manifest a partir de CSV
    # ⚠️ EDITAR: ajustar conforme estrutura do seu dataset
    !stage1_5 dataset build-manifest \
        data/metadata.csv \
        --audio-root data/wav \
        --output data/manifest.jsonl
    
    print("✅ Dataset baixado e manifest criado!")

# Verificar dataset
!head -3 data/manifest.jsonl
!wc -l data/manifest.jsonl

---

## 🔬 Parte 3: Extração de Features

Extrai features de múltiplas representações:
- Acoustic (MFCC, F0, speaking rate)
- ECAPA (speaker embeddings)
- SSL (WavLM)
- **Backbone (Qwen3-TTS)** ← foco principal


In [ ]:
%%time
# 3.1 Acoustic features (~ 2 min para 100 utterances)
print("🎵 Extraindo features acústicas...\n")

!stage1_5 features acoustic \
    data/manifest.jsonl \
    artifacts/features/acoustic \
    --sample-rate 16000

print("\n✅ Acoustic features extraídas!")
!ls -lh artifacts/features/acoustic/ | head -5

In [ ]:
%%time
# 3.2 ECAPA embeddings (~ 3 min para 100 utterances)
print("🎙️ Extraindo ECAPA embeddings...\n")

!stage1_5 features ecapa \
    data/manifest.jsonl \
    artifacts/features/ecapa \
    --device cuda

print("\n✅ ECAPA embeddings extraídos!")
!ls -lh artifacts/features/ecapa/ | head -5

In [ ]:
%%time
# 3.3 SSL features (~ 10 min para 100 utterances)
print("🌐 Extraindo SSL features (WavLM)...\n")

!stage1_5 features ssl \
    data/manifest.jsonl \
    artifacts/features/ssl \
    --model wavlm_large \
    --layers 0 6 12 18 24 \
    --device cuda \
    --torch-dtype bfloat16 \
    --pooling mean

print("\n✅ SSL features extraídas!")
!ls -lh artifacts/features/ssl/ | head -5

In [ ]:
# Limpar cache CUDA antes de carregar backbone
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

print(f"💾 VRAM livre: {torch.cuda.mem_get_info()[0] / 1e9:.1f}GB")

In [ ]:
%%time
# 3.4 Backbone features (Qwen3-TTS) - ETAPA PRINCIPAL
# ~ 30-60 min para 100 utterances (depende da GPU)

print("🧠 Extraindo features do backbone (Qwen3-TTS)...\n")
print("⚠️  Esta etapa pode demorar. Progresso será mostrado.\n")

# Checkpoint Qwen3-TTS
CHECKPOINT = "Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice"

# Layers para extrair
LAYERS = [
    "text_encoder_out",
    "decoder_block_04",
    "decoder_block_08",
    "decoder_block_12",
    "pre_vocoder"
]

!stage1_5 features backbone \
    data/manifest.jsonl \
    data/texts.json \
    artifacts/features/backbone \
    --checkpoint {CHECKPOINT} \
    --layers {" ".join(LAYERS)} \
    --device cuda \
    --dtype bfloat16 \
    --attn-implementation flash-attn3 \
    --generation-mode custom_voice \
    --generation-language Portuguese \
    --generation-speaker ryan \
    --generation-max-new-tokens 256 \
    --pooling mean \
    --strict False

print("\n✅ Backbone features extraídas!")
print("\n📊 Inspecionando features:")
!ls -lh artifacts/features/backbone/ | head -5

# Verificar uma feature
import numpy as np
sample_feat = list(Path("artifacts/features/backbone").glob("*.npz"))[0]
data = np.load(sample_feat)
print(f"\n📦 Sample feature: {sample_feat.name}")
print(f"   Layers: {data.files}")
print(f"   Shapes: {dict((k, data[k].shape) for k in data.files)}")

---

## 🎯 Parte 4: Treinamento de Probes

Treina classificadores lineares para:
- **Accent separability** (speaker-disjoint)
- **Speaker identification**
- **Leakage tests** (A→S, S→A)
- **Text robustness**


In [ ]:
# Limpar cache CUDA
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

print(f"💾 VRAM livre: {torch.cuda.mem_get_info()[0] / 1e9:.1f}GB")

In [ ]:
%%time
# Rodar pipeline completo de probes + análise
# ~ 10-20 min

print("🔬 Executando pipeline de probes e análise...\n")

!stage1_5 run config/stage1_5.yaml

print("\n✅ Pipeline completo!")

---

## 📊 Parte 5: Análise de Resultados

Visualiza métricas, heatmaps e decisão GO/NOGO.


In [ ]:
# Mostrar métricas
import pandas as pd

metrics = pd.read_csv("artifacts/analysis/metrics.csv")

print("📊 MÉTRICAS DE SEPARABILIDADE\n")
print("=" * 80)

# Mostrar top 5 layers por accent F1
top5 = metrics.sort_values("accent_f1", ascending=False).head(5)

print("\n🏆 Top 5 Layers por Accent F1 (speaker-disjoint):\n")
print(top5[[
    "label",
    "accent_f1",
    "speaker_acc",
    "leakage_a2s",
    "leakage_s2a",
    "accent_text_drop"
]].to_string(index=False))

print("\n" + "=" * 80)

# Estatísticas gerais
print("\n📈 Estatísticas Gerais:\n")
print(f"   - Best Accent F1: {metrics['accent_f1'].max():.3f}")
print(f"   - Best Speaker Acc: {metrics['speaker_acc'].max():.3f}")
print(f"   - Min Leakage A→S: {metrics['leakage_a2s'].min():.3f}")
print(f"   - Min Text Drop: {metrics['accent_text_drop'].min():.3f}")

# Comparar backbone vs SSL vs acoustic
print("\n🔍 Comparação por Tipo:\n")
for feature_type in ["backbone", "ssl", "acoustic", "ecapa"]:
    subset = metrics[metrics["label"].str.startswith(feature_type)]
    if len(subset) > 0:
        print(f"   {feature_type.upper():12s}: F1={subset['accent_f1'].max():.3f} (max)")

In [ ]:
# Visualizar heatmaps
from IPython.display import Image, display

print("🎨 HEATMAPS\n")
print("=" * 80)

heatmaps = [
    ("Accent F1", "artifacts/analysis/figures/accent_f1.png"),
    ("Leakage", "artifacts/analysis/figures/leakage.png"),
    ("Text Robustness", "artifacts/analysis/figures/accent_text_robustness.png"),
]

for title, path in heatmaps:
    if os.path.exists(path):
        print(f"\n📊 {title}:\n")
        display(Image(filename=path))
    else:
        print(f"⚠️  {title} não encontrado em {path}")

In [ ]:
# Mostrar relatório final
print("📄 RELATÓRIO FINAL\n")
print("=" * 80)

with open("report/stage1_5_report.md", "r") as f:
    report = f.read()

# Extrair decisão
import re
decision_match = re.search(r"\*\*Decision:\*\* (.+)", report)
rationale_match = re.search(r"\*\*Rationale:\*\* (.+)", report)

if decision_match:
    decision = decision_match.group(1)
    print(f"\n🎯 DECISÃO: {decision}\n")

if rationale_match:
    rationale = rationale_match.group(1)
    print(f"📝 Justificativa: {rationale}\n")

print("\n" + "=" * 80)
print("\n📄 Relatório completo salvo em: report/stage1_5_report.md")
print("\n💾 Para baixar, use o explorador de arquivos do Colab (ícone de pasta à esquerda)")

---

## 💾 Parte 6: Download de Resultados

Baixa todos os artefatos gerados.


In [ ]:
%%time
# Empacotar resultados
print("📦 Empacotando resultados...\n")

!zip -r -q stage1_5_results.zip \
    report/ \
    artifacts/analysis/ \
    artifacts/probes/ \
    config/

# Estatísticas do arquivo
import os
size_mb = os.path.getsize("stage1_5_results.zip") / 1e6
print(f"✅ Resultados empacotados: stage1_5_results.zip ({size_mb:.1f} MB)")

# Download via Colab
from google.colab import files
print("\n📥 Baixando arquivo...")
files.download("stage1_5_results.zip")

### (Opcional) Sincronizar com Google Drive


In [ ]:
# Sincronizar com Google Drive (opcional)
from google.colab import drive

drive.mount('/content/drive')

# Copiar resultados
!mkdir -p /content/drive/MyDrive/stage1_5_results
!cp -r report/ /content/drive/MyDrive/stage1_5_results/
!cp -r artifacts/analysis/ /content/drive/MyDrive/stage1_5_results/
!cp stage1_5_results.zip /content/drive/MyDrive/

print("✅ Resultados sincronizados com Google Drive!")
print("📂 Localização: MyDrive/stage1_5_results/")

---

## 🧹 Parte 7: Limpeza (Opcional)

Libera espaço em disco e VRAM.


In [ ]:
# Limpar features intermediárias (manter apenas análise)
!rm -rf artifacts/features/
!rm -rf data/wav/

# Limpar cache Python
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

# Estatísticas de espaço
!df -h /content

print("\n✅ Limpeza concluída!")

---

## 🔧 Troubleshooting

### Erro: CUDA out of memory

**Solução 1**: Reduzir batch size
```python
# Editar config/stage1_5.yaml
# batch_size: 4  # reduzir para 2 ou 1
```

**Solução 2**: Usar float16 em vez de bfloat16
```bash
!stage1_5 features backbone ... --dtype float16
```

**Solução 3**: Processar em lotes menores
```bash
# Dividir manifest em chunks
!split -l 20 data/manifest.jsonl data/manifest_chunk_
# Processar cada chunk separadamente
```

### Erro: Flash Attention não disponível

```bash
# Remover flag flash-attn3
!stage1_5 features backbone ... --attn-implementation eager
```

### Erro: Qwen-TTS não instalado

```bash
!pip install -U qwen-tts
```

### Verificar logs detalhados

```python
import logging
logging.basicConfig(level=logging.DEBUG)
```

---

## 📚 Referências

- **PRD**: `/content/stage1_5/PRD.md`
- **GATE**: `/content/stage1_5/GATE_1_5.md`
- **README**: `/content/stage1_5/README.md`
- **Fixes**: `/mnt/user-data/outputs/IMPLEMENTATION_GUIDE.md`

---

## 🎉 Fim!

Pipeline Stage 1.5 completo. Próximos passos:

1. ✅ Analisar `report/stage1_5_report.md`
2. ✅ Verificar decisão GO/NOGO
3. ✅ Se GO → prosseguir para Stage 2 (LoRA training)
4. ✅ Se NOGO → ajustar dataset/backbone conforme recomendado

**Dúvidas?** Consulte a documentação ou abra uma issue no GitHub.
